# Voting Classifier con los mejores 3 modelos obtenidos

Como modelo adicional a los considerados, utilizaremos un modelo ensamblado considerando los 3 mejores modelos obtenidos al momento de entrenamiento. Así, en este notebook utilizaremos el voto de los 3 mejores modelos para obtener la predicción de las fallas y así posteriormente, evaluar el desempeño para entrar en comparación con los demás.

In [1]:
import os
import json
import numpy as np
import pandas as pd
import datetime as dt
from sklearn.externals import joblib
from sklearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler

C:\Users\pasal\Anaconda3\envs\PI2\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
### Realizamos el cambio de directoroi de trabajo al "Directorio Base" que se
current_dir = os.getcwd()
base_path = os.path.dirname(current_dir)

os.chdir(base_path)

In [3]:
from sklearn.externals import joblib
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
from mlxtend.classifier import EnsembleVoteClassifier

In [4]:
import scripts.funciones as funciones
from scripts.clase_model.modelo import Modelo

### Organización de la información

Importamos la información incluyendo tanto la ventana de tiempo de entrenamiento como la de prueba. Para esto, organizamos la información acorde a las caracteristicas del modelo entrenado

In [5]:
d_ini = dt.datetime(2017,6,1)
d_fin = dt.datetime(2020,1,1)

In [6]:
version = 'verFinal'    
mod_version = funciones.carga_model(base_path, f'models/{version}', version)

if 'model' in mod_version:
    print("Model loaded")
    mod = mod_version['model'].steps[0][1]
    model_sel = mod_version['model'].steps[1][1]
else:
    print("No model found")
    

data = funciones.read_clima_accidentes(d_ini, d_fin, poblado = True)
data_org = funciones.organizar_data_infoClima(data)

### agregamos la informacion relacionada a la cantidad de accidentes ocurridas
### en las ultimas X horas

### En caso que se considere acumulado de fallas, realiza la validacion
### si el modelo entrenado tiene la frecuencia utilizada
try:
    senales = mod.freq1
    freq2 = mod.freq2
except Exception as e:
    print(f'Problemas con las frecuencias de las senales {e}')
    freq1 = '1H'
    freq2 = '1H'

d_ini_acc = d_ini - dt.timedelta(days = int(freq2.replace('D', '')))  ### freq mayor
raw_accidentes = funciones.read_accidentes(d_ini_acc, d_fin)
for fresen in senales:
    data_org = funciones.obtener_accidentes_acumulados(data_org, 
                                                        raw_accidentes,
                                                       freq = fresen)

data_org['poblado'] = data_org['BARRIO']
data_org= pd.get_dummies(data_org, columns=['poblado'])

Model loaded


Realizamos la división del conjunto de datos en entrenamiento, validación y prueba.

In [7]:
d_ini_train = dt.datetime(2017,6,1)
d_fin_train = dt.datetime(2019,8,1)  

d_ini_test = dt.datetime(2019,8,1)
d_fin_test = dt.datetime(2020,1,1)

train = (data_org['TW']>= d_ini_train) & (data_org['TW']< d_fin_train)
test = (data_org['TW']>= d_ini_test) & (data_org['TW']< d_fin_test)
    
X_train = data_org[train].drop(columns = ['TW','BARRIO','Accidente','summary']).reset_index(drop = True)
Y_train = data_org[train]['Accidente'].reset_index(drop = True)

X_test = data_org[test].drop(columns = ['TW','BARRIO','Accidente','summary']).reset_index(drop = True)
Y_test = data_org[test]['Accidente'].reset_index(drop = True)

X_train = X_train[mod.cols_order]
X_test = X_test[mod.cols_order]

X_train2, X_val, Y_train2, Y_val = train_test_split(X_train, 
                                        Y_train,
                                        stratify = Y_train,
                                        test_size = 0.2,
                                        random_state = 42)

# Cargar Modelos Entrenamos

Cargamos en memoria los 3 mejores modelos obtenidos, y los ensamblamos en un clasificador que los tomará y en base a la predicción de los 3 modelos, entregará una única predicción de ocurrencia del accidente

In [8]:
mod1 = funciones.carga_model(base_path, f'models/{version}', 'nn_20200513_0827')['model']
mod2 = funciones.carga_model(base_path, f'models/{version}', 'logistic_20200513_0827')['model']
mod3 = funciones.carga_model(base_path, f'models/{version}', 'xtree_20200513_0827')['model']

In [9]:
VotingClassifier = EnsembleVoteClassifier(clfs=[mod1, mod2, mod3], weights=[1,1,1], refit=False)
VotingClassifier.fit(X_train, Y_train)

EnsembleVoteClassifier(clfs=[Pipeline(memory=None,
                                      steps=[('scaler',
                                              StandardScaler(copy=True,
                                                             with_mean=True,
                                                             with_std=True)),
                                             ('nn',
                                              MLPClassifier(activation='identity',
                                                            alpha=0.05,
                                                            batch_size='auto',
                                                            beta_1=0.9,
                                                            beta_2=0.999,
                                                            early_stopping=False,
                                                            epsilon=1e-08,
                                                            hidden_layer_size

Obtenemos las métricas del modelo en el conjunto de validación

In [10]:
X_val = pd.read_csv('data/validation.csv')
Y_val = X_val['Accidente']

X_val = X_val[mod.cols_order]

In [11]:
preds = VotingClassifier.predict(X_val)
proba = VotingClassifier.predict_proba(X_val)

In [12]:
ROC = metrics.roc_auc_score(Y_val, proba[:,1])
PR = funciones.precision_recall_auc_score(Y_val, proba[:,1])
bAccuracy = metrics.balanced_accuracy_score(Y_val,preds) 
fScore = metrics.f1_score(Y_val,preds) 
precision = metrics.precision_score(Y_val,preds)
recall = metrics.recall_score(Y_val,preds)

print(f'ROC-AUC: {ROC}')
print(f'PR-AUC: {PR}')
print(f'bAccuracy: {bAccuracy}')
print(f'fScore: {fScore}')
print(f'precision: {precision}')
print(f'recall: {recall}')

ROC-AUC: 0.7844021022705707
PR-AUC: 0.060799060893915374
bAccuracy: 0.666029051685836
fScore: 0.11473463049237234
precision: 0.06569724866617847
recall: 0.45244956772334294


In [13]:
### Guardo el modelo de voting
mod_pipe = Pipeline([('procesador', mod),
                    ('modelo', VotingClassifier)])
    
path_best_mod = os.path.join(f'{base_path}/models/{version}', f"{version}_voting.sav")

joblib.dump(mod_pipe, path_best_mod) 

['C:\\Users\\pasal\\Google Drive\\EAFIT\\Ciencia de Datos y Analitica\\02 Segundo Semestre\\Proyecto Integrador\\Proyecto-Integrador-2/models/verFinal\\verFinal_voting.sav']

# Evaluación del modelo en el conjunto de prueba

Obtenemos las métricas del modelo en el conjunto de prueba

In [ ]:
preds = VotingClassifier.predict(X_test)
proba = VotingClassifier.predict_proba(X_test)

In [ ]:
ROC = metrics.roc_auc_score(Y_test, proba[:,1])
bAccuracy = metrics.balanced_accuracy_score(Y_test,preds) 
fScore = metrics.f1_score(Y_test,preds) 
precision = metrics.precision_score(Y_test,preds)
recall = metrics.recall_score(Y_test,preds)

print(f'ROC-AUC: {ROC}')
print(f'bAccuracy: {bAccuracy}')
print(f'fScore: {fScore}')
print(f'precision: {precision}')
print(f'recall: {recall}')

# Lift Plot

In [ ]:
%matplotlib notebook

In [ ]:
import scikitplot as skplt
import matplotlib.pyplot as plt

In [ ]:
mod1 = funciones.carga_model(base_path, f'models/{version}', 'nn_20200505_2014')['model']

Y_test_pred_ = mod1.predict_proba(X_test)[:]
skplt.metrics.plot_cumulative_gain(Y_test, Y_test_pred_)
plt.show()
skplt.metrics.plot_lift_curve(Y_test, Y_test_pred_)
plt.show()

In [ ]:
#X_test
X_test['TW'] = data_org[test]['TW'].values
X_test['BARRIO'] = data_org[test]['BARRIO'].values
X_test['Probabilidad'] = Y_test_pred_[:,1]
X_test['Prediccion_Accidente'] = (X_test['Probabilidad']>=0.5).astype(int)

filt = X_test['TW']== dt.datetime(2019,12,23,13)

In [ ]:
X_test[filt][['TW','BARRIO','Probabilidad','Prediccion_Accidente']].reset_index(drop = True)